In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
JAR_PATH = "/home/khanhpluto/khanhtv/test_spark/zxc/jars_dir/spark-sql-kafka-0-10_2.12-3.2.1.jar,/home/khanhpluto/khanhtv/test_spark/zxc/jars_dir/kafka-clients-3.1.0.jar"

In [ ]:
spark = SparkSession.builder.appName("Spark kafka") \
    .config("spark.jars", JAR_PATH) \
    .config("spark.executor.extraClassPath", JAR_PATH) \
    .config("spark.executor.extraLibrary", JAR_PATH) \
    .config("spark.driver.extraClassPath", JAR_PATH) \
    .getOrCreate()

23/01/10 11:02:15 WARN Utils: Your hostname, khanhpluto-Z10PE-D8-WS resolves to a loopback address: 127.0.1.1; using 192.168.9.45 instead (on interface eno1)
23/01/10 11:02:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/01/10 11:02:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/10 11:02:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/01/10 11:02:16 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [ ]:
spark.sparkContext.setLogLevel("ERROR")

In [5]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "192.168.9.69:9092") \
  .option("subscribe", "topic1") \
  .load()

In [6]:
transaction_detail_df1 = df.selectExpr("CAST(value AS STRING)", "timestamp")

In [7]:
transaction_detail_df1.printSchema()

root
 |-- value: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [13]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [14]:
transaction_detail_schema = StructType() \
    .add("transaction_id", StringType()) \
    .add("transaction_card_type", StringType()) \
    .add("transaction_amount", StringType()) \
    .add("transaction_datetime", StringType())

In [19]:
transaction_detail_df2 = transaction_detail_df1\
    .select(from_json(col("value"), transaction_detail_schema).alias("transaction_detail"), "timestamp")

In [20]:
transaction_detail_df3 = transaction_detail_df2.select("transaction_detail.*", "timestamp")

In [21]:
transaction_detail_df4 = transaction_detail_df3.groupBy("transaction_card_type")\
    .agg({'transaction_amount': 'sum'}).select("transaction_card_type", \
    col("sum(transaction_amount)").alias("total_transaction_amount"))

In [23]:
print("Printing Schema of transaction_detail_df4: ")
transaction_detail_df4.printSchema()

Printing Schema of transaction_detail_df4: 
root
 |-- transaction_card_type: string (nullable = true)
 |-- total_transaction_amount: double (nullable = true)



In [24]:
transaction_detail_df5 = transaction_detail_df4.withColumn("key", lit(100))\
                                                .withColumn("value", concat(lit("{'transaction_card_type': '"), \
                                                col("transaction_card_type"), lit("', 'total_transaction_amount: '"), \
                                                col("total_transaction_amount").cast("string"), lit("'}")))

In [25]:
print("Printing Schema of transaction_detail_df5: ")
transaction_detail_df5.printSchema()

Printing Schema of transaction_detail_df5: 
root
 |-- transaction_card_type: string (nullable = true)
 |-- total_transaction_amount: double (nullable = true)
 |-- key: integer (nullable = false)
 |-- value: string (nullable = true)



In [26]:
trans_detail_write_stream = transaction_detail_df5 \
    .writeStream \
    .trigger(processingTime='1 seconds') \
    .outputMode("update") \
    .option("truncate", "false")\
    .format("console") \
    .start()

In [31]:
trans_detail_write_stream_1 = transaction_detail_df5 \
    .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "192.168.9.69:9092") \
    .option("topic", "topic1") \
    .trigger(processingTime='1 seconds') \
    .outputMode("update") \
    .option("checkpointLocation", "./py_checkpoint") \
    .start()

trans_detail_write_stream.awaitTermination()

print("PySpark Structured Streaming with Kafka Demo Application Completed.")

StreamingQueryException: org/apache/spark/kafka010/KafkaConfigUpdater
=== Streaming Query ===
Identifier: [id = 550ee2e2-b2a9-4300-a5c5-e5ff278db498, runId = d21b36df-177d-4f6c-a0ed-4544b44340ec]
Current Committed Offsets: {}
Current Available Offsets: {}

Current State: INITIALIZING
Thread State: RUNNABLE

In [32]:
customer_data_df=spark.read.format("csv") \
    .option("header", "true") \
    .option("delimiter", ",") \
    .load("data/retails.csv")

In [34]:
customer_data_df.show()

+---------+---------+--------------------+--------+------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity| InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/10 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/10 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/10 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/10 8:26|     4.25|     17850|United

In [35]:
stream_detail_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "192.168.9.69:9092") \
    .option("subscribe", "topic1") \
    .option("startingOffsets", "latest") \
    .load()